ドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


パラメータ定義

In [ ]:
from PIL import Image,ImageFile
import os,glob
import numpy as np

n_categories=2
test_batch_size = 1
image_size = 224
label_lists = ["me","notme"]
val_dataset = "./drive/My Drive/me_or_notme/val/"
test_dataset = "./drive/My Drive/me_or_notme/test/"
file_name = 'vgg16_me_or_notme_file'
save_model_path = "./drive/My Drive/me_or_notme/model/" 
result_path = "./drive/My Drive/me_or_notme/results/"

格納するリストを定義

In [ ]:
    ok = []
    no = []
    answer_ok = []
    answer_no = []
    pred_labels = []
    test_labels = []
    filenames = []
    ok_filenames = []
    no_filenames = []
    no_pred = []

モデルのロード

In [ ]:
from keras.models import load_model

model = load_model( "./drive/My Drive/me_or_notme/model/vgg16_last_model.h5")
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

テストgeneratorの作成

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
if n_categories == 2:
    select_class_mode = 'binary'
else:
        # 多分類のため categorical
    select_class_mode ='categorical'

test_generator = test_datagen.flow_from_directory(
    test_dataset,
    target_size=(image_size,image_size),
    batch_size=1,
    class_mode = select_class_mode)

Found 10 images belonging to 2 classes.


定義の確認

In [ ]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
test_loss, test_acc = model.evaluate_generator(
    test_generator, steps=STEP_SIZE_TEST, verbose=1)
print('test loss: ', test_loss)
print('test acc : ', test_acc)
print(test_generator.class_indices)

Instructions for updating:
Please use Model.evaluate, which supports generators.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
test loss:  0.006513493543025106
test acc :  1.0
{'me': 0, 'notme': 1}


テストの実行

In [ ]:
from keras.preprocessing import image

for one_label in label_lists:
    one_label_path = test_dataset + one_label + '/'
    for one_file in os.listdir(one_label_path):
        # 教師データ
        test_labels.append(test_generator.class_indices[one_label])
        one_image_file_path = os.path.join(one_label_path, one_file)
        print(one_image_file_path)
        img = image.load_img(one_image_file_path,
                                target_size=(image_size, image_size))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis=0)

        # このモデルの訓練に入力が次の前処理されているので注意
        img_tensor /= 255.
        pred = model.predict(img_tensor, batch_size=1, verbose=0)
        print(pred)
        pred_format = []
        score = np.max(pred)
        pred_label = np.argmax(pred)
        print(pred_label)

        # 予測したラベル
        pred_labels.append(pred_label)
        filenames.append(one_image_file_path)

        if test_generator.class_indices[one_label] == pred_label:
            ok.append(test_generator.class_indices[one_label])
            answer_ok.append(test_generator.class_indices[one_label])
            ok_filenames.append(one_image_file_path)
        else:
            no_pred.append(
                [k for k, v in test_generator.class_indices.items() if v == pred_label][0])
            no.append(test_generator.class_indices[one_label])
            answer_no.append([k for k, v in test_generator.class_indices.items(
            ) if v == test_generator.class_indices[one_label]][0])
            no_filenames.append(one_image_file_path)


./drive/My Drive/me_or_notme/test/me/me1.jpg
[[0.00239617]]
0
./drive/My Drive/me_or_notme/test/me/me2.jpg
[[0.01821291]]
0
./drive/My Drive/me_or_notme/test/me/me3.jpg
[[0.00428218]]
0
./drive/My Drive/me_or_notme/test/me/me4.jpg
[[0.00092939]]
0
./drive/My Drive/me_or_notme/test/me/me5.jpg
[[0.00332496]]
0
./drive/My Drive/me_or_notme/test/notme/notme1.jpg
[[0.9904763]]
0
./drive/My Drive/me_or_notme/test/notme/notme2.jpg
[[0.99210155]]
0
./drive/My Drive/me_or_notme/test/notme/notme3.jpg
[[0.9926946]]
0
./drive/My Drive/me_or_notme/test/notme/notme4.jpg
[[0.9943347]]
0
./drive/My Drive/me_or_notme/test/notme/notme5.jpg
[[0.99472237]]
0


分析用のcsvを出力

In [ ]:
import pandas as pd
import shutil
import sys

# 正解数カウント
print('total ok data:{}'.format(len(ok)))
for name, index in test_generator.class_indices.items():
    n = ok.count(index)
    print('{:20} : {:8}'.format(name, n))

#  不正解数カウント
print('total no data:{}'.format(len(no)))
for name, index in test_generator.class_indices.items():
    n = no.count(index)
    print('{:20} : {:8}'.format(name, n))
print(test_generator.class_indices.keys())

# ### 正解数と不正解数と正解率
df_result = pd.DataFrame(index=test_generator.class_indices.keys(), columns=[
    'Correct', 'Incorrect', 'Recall', 'TotalData'])
for name, index in test_generator.class_indices.items():
    df_result['Correct'][name] = ok.count(index)
    df_result['Incorrect'][name] = no.count(index)
    df_result['TotalData'][name] = total = ok.count(
        index) + no.count(index)
    if ok.count(index) == 0:
        df_result['Recall'][name] = 0
    else:
        df_result['Recall'][name] = ok.count(index) / total

# 正解数,不正解のカウント
df_result_total = pd.DataFrame(index=['Total'], columns=[
    'Correct', 'Incorrect', 'Recall', 'TotalData'])
df_result_total['Correct'] = len(ok)
df_result_total['Incorrect'] = len(no)
df_result_total['Recall'] = (len(ok) / (len(ok) + len(no)))
df_result_total['TotalData'] = len(ok) + len(no)
df_result = pd.concat([df_result, df_result_total])
print(df_result)

# 結果保存用のディレクトリを保存
if not os.path.exists(result_path):
    os.mkdir(result_path)
    print('make dir')
test_result_path  = result_path + 'test_result/'
if not os.path.exists(test_result_path):
    os.mkdir(test_result_path)
    print('make dir')
save_detail_path = test_result_path + 'Details_data/'
if not os.path.exists(save_detail_path):
    os.mkdir(save_detail_path)
    print('make dir')
df_result.to_csv(save_detail_path +"result_data.csv", 
                 index=test_generator.class_indices.keys())

# 適合率・再現率の計算とcsv出力
if len(test_generator.class_indices.items()) == 2:
    #"Accuracy", "Precision","Recall", "F-measure"])
    tp = df_result['Correct'][label_lists[0]]
    tn = df_result['Correct'][label_lists[1]]
    fn = df_result['Incorrect'][label_lists[0]]
    fp = df_result['Incorrect'][label_lists[1]]
    acc = (tp + tn) / (tp + tn + fn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = (2*recall * precision) / (recall + precision)
    confusion_matrix = pd.DataFrame([acc, precision, recall, f_measure], index=["Accuracy", "Precision", "Recall", "F-measure"] )
    confusion_matrix.to_csv(save_detail_path+ "confusion_matrix.csv")
    print(confusion_matrix)
    confident_and_lebel = pd.DataFrame({"Filename": filenames,
                            "Answer": test_labels,
                            "Predictions": pred_labels,
                            "1st": score})
    
confident_and_lebel.to_csv(save_detail_path + "confident_and_lebel.csv", index=False)
#top5_error_csv(save_detail_path + "confident_and_lebel.csv", test_generator.class_indices.keys())
ok_results = pd.DataFrame({"Filename": ok_filenames,
                            "Answer": answer_ok,
                            "Predictions": ok})
ok_results.to_csv(save_detail_path + "ok_results.csv", index=False)

no_results = pd.DataFrame({"Filename": no_filenames,
                            "Answer": answer_no,
                            "Predictions": no_pred})
no_results.to_csv(save_detail_path + "no_results.csv", index=False)
)





total ok data:5
me                   :        5
notme                :        0
total no data:5
me                   :        0
notme                :        5
dict_keys(['me', 'notme'])
      Correct Incorrect Recall TotalData
me          5         0      1         5
notme       0         5      0         5
Total       5         5    0.5        10
                  0
Accuracy   0.500000
Precision  0.500000
Recall     1.000000
F-measure  0.666667


失敗画像の保存

In [ ]:
miss_image_dir = result_path + 'miss_image/'
if os.path.exists(miss_image_dir):
    shutil.rmtree(miss_image_dir)
os.mkdir(miss_image_dir)
for no_filename in no_filenames:
    filename_spilit = no_filename.split("/")
    label_name = filename_spilit[-2]
    image_name = filename_spilit[-1]
    miss_image_path = miss_image_dir + label_name + '/' + image_name
    if not os.path.exists(miss_image_dir+ label_name + '/'):
        os.mkdir(miss_image_dir + label_name + '/')
    shutil.copy(no_filename, miss_image_path

失敗画像のヒートマップの保存準備

In [ ]:
heatmap_image_dir = result_path +'miss_heatmap_image/'
if os.path.exists(heatmap_image_dir):
    shutil.rmtree(heatmap_image_dir)
os.mkdir(heatmap_image_dir)

save_dir_path = result_path +'concat_missimage/'
if os.path.exists(save_dir_path):
    shutil.rmtree(save_dir_path)
os.mkdir(save_dir_path)

ヒートマップ作成用の関数の定義

In [ ]:
def make_image_heatmap(name, model, heatmap_image_dir):
    
    bear_image_path = name
    img = image.load_img(bear_image_path, target_size=(image_size, image_size))
    #img_tensor.shape = (224, 224, 3)   
    img_tensor = image.img_to_array(img)
    #img_tensor.shape = (1, 224, 224, 3)   
    img_tensor = np.expand_dims(img_tensor, axis=0)

    # モデルの訓練時と同じ方法で前処理
    img_tensor /= 255.
    preds = model.predict(img_tensor)

    model_output = model.output[:, np.argmax(preds[0])]
    #最後の畳み込み層を取得
    last_conv_layer = model.get_layer('block5_conv3')
    grads = K.gradients(model_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function(
        [model.input], [pooled_grads, last_conv_layer.output[0]])
    
    pooled_grads_val, conv_output_val = iterate([img_tensor])

    for i in range(512):
        #conv_output_val[0,:,:, i] = conv_output_val[:,:, i] * pooled_grads_val[i]
        conv_output_val[:,:, i] *= pooled_grads_val[i]

    cam = np.mean(conv_output_val, axis=-1)
    # ヒートマップの後処理
    cam = np.maximum(cam, 0)
    cam /= np.max(cam)

    # cv2を使って元画像を読み込む
    img = cv2.imread(bear_image_path)
    
    # 元の画像と同じサイズになるようにヒートマップのサイズを変更
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))

    # ヒートマップをRGBに変換
    cam = np.uint8(255 * cam)
    # ヒートマップを元画像に適用
    cam = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    # 0.4はヒートマップの強度係数
    superimposed_img = cam * 0.4 + img
    tmp = name.split('/')
    # ファイル名を決める
    file_name = tmp[-1]
    dir_name = tmp[-2]
    # 画像を保存
    heatmap_save_dir = heatmap_image_dir + tmp[-2] + '/'
    heatmap_file_name = heatmap_save_dir + 'heatmap_' + file_name
    if not os.path.exists(heatmap_save_dir):
        os.makedirs(heatmap_save_dir)
    cv2.imwrite(heatmap_file_name, superimposed_img)

    return file_name, dir_name, heatmap_file_name

失敗画像のヒートマップの作成と保存

In [ ]:
from tensorflow.python.keras import backend as K 
import tensorflow.compat.v1 as tf
import cv2

# このプログラムコード以降はどうしても動かないところがあるため1系に変更
tf.disable_v2_behavior()

no_num = 1
for name in no_filenames:
    #sys.stdout.write("\r make no heatmap image... = {}/{}".format(no_num,len(no_filenames)))
    #sys.stdout.flush()

    file_name, dir_name, heatmap_file_name = make_image_heatmap(name, model, heatmap_image_dir)

    im1 = Image.open(name)
    im2 = Image.open(heatmap_file_name)

    tmp = heatmap_file_name.split('/')
    
    save_dir_path = result_path +'concat_missimage/' + dir_name + '/'
    if not os.path.exists(save_dir_path):
        os.mkdir(save_dir_path)
    get_concat_h(im1, im2).save(save_dir_path + 'miss_' + file_name)

    no_num += 1
print("finished")

finished


成功画像のヒートマップの作成準備

In [ ]:
heatmap_image_dir = result_path +'ok_heatmap_image/'
if not os.path.exists(heatmap_image_dir):
    # shutil.rmtree(heatmap_image_dir)
    os.mkdir(heatmap_image_dir)

save_dir_path = result_path + 'concat_okimage/'
if not  os.path.exists(save_dir_path):
    os.mkdir(save_dir_path)
ok_filenames_num = len(ok_filenames)
#random.shuffle(ok_filenames)
ok_num = 0

成功画像のヒートマップの作成

In [ ]:
for name in ok_filenames:
    if (ok_filenames_num >= 0):

        file_name, dir_name, heatmap_file_name = make_image_heatmap(name, model, heatmap_image_dir)

        im1 = Image.open(name)
        im2 = Image.open(heatmap_file_name)

        save_dir_path = result_path +'concat_okimage/' + dir_name + '/'
        if not os.path.exists(save_dir_path):
            os.mkdir(save_dir_path)
        get_concat_h(im1, im2).save(save_dir_path + 'ok_' + file_name)
        ok_num += 1
print("finished")

finished
